# Identifying Safe Loans With Binary Decision Trees

In [38]:
import graphlab

##Loading dataset:-

In [39]:
loans = graphlab.SFrame('lending-club-data.gl/')

In [40]:
loans

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade
1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2
1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4
1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5
1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1
1075269,1311441,5000,5000,5000,36 months,7.9,156.46,A,A4
1072053,1288686,3000,3000,3000,36 months,18.64,109.43,E,E1
1071795,1306957,5600,5600,5600,60 months,21.28,152.39,F,F2
1071570,1306721,5375,5375,5350,60 months,12.69,121.45,B,B5
1070078,1305201,6500,6500,6500,60 months,14.65,153.45,C,C3
1069908,1305008,12000,12000,12000,36 months,12.69,402.54,B,B5


##Labelling +1 for a safe loan, and -1 for a risky loan.

In [41]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')

In [42]:
features = ['grade',             
            'term',               
            'home_ownership',     
            'emp_length',         
           ]
target = 'safe_loans'
loans = loans[features + [target]]
loans

grade,term,home_ownership,emp_length,safe_loans
B,36 months,RENT,10+ years,1
C,60 months,RENT,< 1 year,-1
C,36 months,RENT,10+ years,1
C,36 months,RENT,10+ years,1
A,36 months,RENT,3 years,1
E,36 months,RENT,9 years,1
F,60 months,OWN,4 years,-1
B,60 months,RENT,< 1 year,-1
C,60 months,OWN,5 years,1
B,36 months,OWN,10+ years,1


In [43]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.502236174422
Percentage of risky loans                : 0.497763825578
Total number of loans in our new dataset : 46508


##Since it is binary decision trees : transforming categorical data into binary data using 1-hot encoding.

In [44]:
loans_data = risky_loans.append(safe_loans)
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)
    
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)

The feature columns now look like this:

In [45]:
loans_data

safe_loans,grade.A,grade.B,grade.C,grade.D,grade.E,grade.F,grade.G,term. 36 months,term. 60 months
-1,0,0,1,0,0,0,0,0,1
-1,0,0,0,0,0,1,0,0,1
-1,0,1,0,0,0,0,0,0,1
-1,0,0,1,0,0,0,0,1,0
-1,0,1,0,0,0,0,0,1,0
-1,0,1,0,0,0,0,0,1,0
-1,0,1,0,0,0,0,0,1,0
-1,0,0,1,0,0,0,0,1,0
-1,0,0,0,1,0,0,0,0,1
-1,1,0,0,0,0,0,0,1,0


In [9]:
features = loans_data.column_names()
features.remove('safe_loans') 
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

In [10]:
train_data, validation_set = loans_data.random_split(.8, seed=1)

##Implementing binary decision trees early stopping methods:-

In [11]:
def reached_minimum_node_size(data, min_node_size):
    # Returns True if the number of data points is less than or equal to the minimum node size.
    c=False
    if len(data)<=min_node_size:
        c=True
    return c

In [12]:
def error_reduction(error_before_split, error_after_split):
    # Returns the error before the split minus the error after the split.
    f=float(0)
    f=error_before_split-error_after_split
    return f

In [13]:
def intermediate_node_num_mistakes(labels_in_node):
    # If labels_in_node is empty, returning 0
    if len(labels_in_node) == 0:
        return 0
    cp=0
    cn=0
    # Counting the number of 1's (safe loans)
    for i in labels_in_node:
        if i==1:
            cp=cp+1
        else:
            cn=cn+1
            
    # Returning the number of mistakes that the majority classifier makes.
    if(cp>=cn):
     return cn
    else:
     return cp

##Best feature to split on:-

In [14]:
def best_splitting_feature(data, features, target):
    
    best_feature = None # Keeping track of the best feature 
    best_error = 10     # Keeping track of the best error so far 

    num_data_points = float(len(data))  
    
    # Looping through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        right_split = data[data[feature] == 1] 
            
        # Calculating the number of misclassified examples in the left split.
        left_mistakes =  float(intermediate_node_num_mistakes(left_split['safe_loans']))         

        # Calculating the number of misclassified examples in the right split.
        right_mistakes = float(intermediate_node_num_mistakes(right_split['safe_loans']))
            
        # Computeing the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes+right_mistakes)/num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error < best_error:
            best_error=error
            best_feature=feature
        
    
    return best_feature # Returning the best feature we found

In [15]:
def create_leaf(target_values):
    
    # Createing a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf':  True }
    
    # Counting the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, setting the prediction to be the majority class.
    # Storing the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1        
    else:
        leaf['prediction'] = -1     
        
    # Returning the leaf node        
    return leaf 

##Implementing binary decision tree:-

In [16]:
def decision_tree_create(data, features, target,current_depth=0, 
                         max_depth=10,min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Making a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Stopping condition 1 reached. All data points have the same target value."                
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == []:
        print "Stopping condition 2 reached. No remaining features."                
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print "Early stopping condition 1 reached. Reached maximum depth."
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, returning a leaf.
    if  reached_minimum_node_size(data, min_node_size)==True: 
        print "Early stopping condition 2 reached. Reached minimum node size."
        return create_leaf(target_values)
    
    # Finding the best splitting feature
    splitting_feature = best_splitting_feature(data,features, target)
    
    # Splitting on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculating the error before splitting 
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculating the error after splitting 
    left_mistakes =   intermediate_node_num_mistakes(left_split[target])
    right_mistakes =  intermediate_node_num_mistakes(right_split[target])
    error_after_split = float(left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, returning a leaf.
    if  error_reduction(error_before_split, error_after_split)<=min_error_reduction:     
        print "Early stopping condition 3 reached. Minimum error reduction."
        return create_leaf(target_values) 
    
    
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    

    # Creating a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "Creating leaf node."
        return create_leaf(right_split[target])

    # Recursing on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)
    
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [17]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

##Building a tree with
* `max_depth = 6`
* `min_node_size = 100`, 
* `min_error_reduction = 0.0`

In [19]:
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

In [25]:
my_decision_tree = decision_tree_create(train_data, features, 'safe_loans',max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length.n/a. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------------------------
Subtree, depth = 3 (5 data points).
Early stopping condition 2 reached. Reached minimum node size.
-------------------------------------------

##`classify`(): to classify a new point `x` using a given `tree`.

In [22]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # splitting on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate) 

##Identifing whether a loan is safe or not:-

In [23]:
validation_set[0]

{'emp_length.1 year': 0L,
 'emp_length.10+ years': 0L,
 'emp_length.2 years': 1L,
 'emp_length.3 years': 0L,
 'emp_length.4 years': 0L,
 'emp_length.5 years': 0L,
 'emp_length.6 years': 0L,
 'emp_length.7 years': 0L,
 'emp_length.8 years': 0L,
 'emp_length.9 years': 0L,
 'emp_length.< 1 year': 0L,
 'emp_length.n/a': 0L,
 'grade.A': 0L,
 'grade.B': 0L,
 'grade.C': 0L,
 'grade.D': 1L,
 'grade.E': 0L,
 'grade.F': 0L,
 'grade.G': 0L,
 'home_ownership.MORTGAGE': 0L,
 'home_ownership.OTHER': 0L,
 'home_ownership.OWN': 0L,
 'home_ownership.RENT': 1L,
 'safe_loans': -1L,
 'term. 36 months': 0L,
 'term. 60 months': 1L}

In [28]:
classify(my_decision_tree_new, validation_set[0], annotate = True)

Split on term. 36 months = 0
Split on grade.A = 0
At leaf, predicting -1


-1

##Evaluating the model:-

In [30]:
def evaluate_classification_error(tree, data, target):
    # Applying the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x))
    
    f=0
    for i in range(len(data)):
      if prediction[i] != data[target][i]:
        f=f+1 
    acc=float(f)/float(len(data))
    return acc 

In [31]:
evaluate_classification_error(my_decision_tree, validation_set, target)

0.38367083153813014